In [ ]:
!pip install transformers accelerate tiktoken einops transformers_stream_generator==0.0.4 scipy
!pip install auto-gptq optimum
!pip install peft deepspeed

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch

model_id = "Qwen/Qwen-7B-Chat-Int4"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True, torch_dtype=torch.bfloat16, use_flash_attn="True").eval()

In [ ]:
history = None
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a line segment from point (1, 2, 3) to (3, 2, 1) and add it to the viewport. Directly give the code here:'

response, history = model.chat(tokenizer, question, history=history)
print(response)

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!bash /home/ec2-user/SageMaker/Qwen/finetune/finetune_qlora_ds.sh

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model_id = "Qwen/Qwen-14B-Chat-Int4"

tokenizer = AutoTokenizer.from_pretrained("/home/ec2-user/SageMaker/Qwen/checkpoints", trust_remote_code=True)

model = AutoPeftModelForCausalLM.from_pretrained(
    "/home/ec2-user/SageMaker/Qwen/checkpoints",
    device_map="auto",
    trust_remote_code=True
).eval()

In [ ]:
history = None
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a line segment from point (1, 2, 3) to (3, 2, 1) using an original start point and a vector to extend and add it to the viewport. Directly give the code here:'

response, history = model.chat(tokenizer, question, history=history)
print(response)

In [ ]:
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a line segment from point (0, 0, 0) to (1, 1, 1) and add it to the viewport. Directly give the code here:'
response, history = model.chat(tokenizer, question, history=history)
print(response)

In [ ]:
question = 'You are a code assistant of the SSDP platform which is an internal tool used to create 3D scenes. In the SSDP platform, write a function to create a 2d circle with the center(1, 1, 1) , radius 2 and normal vector Z-axis. Directly give the code here:'
response, history = model.chat(tokenizer, question, history=history)
print(response)